### 1) Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма)

In [1]:
from selenium import webdriver          #Основной элемент
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait   # Select - Поле множественного выбора
from selenium.webdriver.common.by import By  #используем Селениум
from selenium.webdriver.common.keys import Keys    #Клавиши клавиатуры
import pprint
from pymongo import MongoClient

link - https://mail.ru/  
login - gb_dz06@mail.ru  
pass - paSSword666!#  

In [2]:
link = "https://mail.ru/"
driver = webdriver.Chrome()   #Движок Хрома
driver.get(link)

In [3]:
assert "Mail.ru: почта" in driver.title
driver.find_element_by_id("mailbox:login").send_keys('gb_dz06')
Select(driver.find_element_by_id('mailbox:domain')).select_by_visible_text("@mail.ru")
driver.find_element_by_id("mailbox:password").send_keys('paSSword666!#')
driver.find_element_by_id("mailbox:submit").click()

wait = WebDriverWait(driver, 15);
wait.until(EC.title_contains("Входящие"));
assert "Входящие" in driver.title

In [4]:
all_items = driver.find_elements_by_xpath("//a[contains(@class,'llc js-tooltip-direction_letter-bottom js-letter-list-item llc_normal')]")

In [5]:
letters_info = []
for item in all_items:
#     print(item.get_attribute('innerHTML'))
    in_letter_link = item.get_attribute('href')
    one_letter_info = item.find_element_by_class_name("llc__content")
    author = one_letter_info.find_element_by_class_name("ll-crpt").get_attribute('title')
    subj = one_letter_info.find_element_by_class_name("ll-sj__normal").text
    date = item.find_element_by_class_name("llc__item_date").get_attribute('title')
    print("From: ", author, "\t\t", date, "\nSubj:", subj, "\nLink:", in_letter_link, "\n\n")
    letters_info.append({"author": author, "date": date, "subj": subj, "link": in_letter_link})

From:  Nikolay Pomazan <pomazannn@gmail.com> 		 Сегодня, 9:30 
Subj: Че не отвечаешь? 
Link: https://e.mail.ru/inbox/1:02380bc5990c721c:0/?back=1&afterReload=1 


From:  Nikolay Pomazan <pomazannn@gmail.com> 		 Сегодня, 9:29 
Subj: А че если фотку прилепить а а а а? 
Link: https://e.mail.ru/inbox/1:12553ef61196f16d:0/?back=1&afterReload=1 


From:  Nikolay Pomazan <pomazannn@gmail.com> 		 Сегодня, 7:55 
Subj: Это тема длинного письма 
Link: https://e.mail.ru/inbox/1:3442578eb55fda88:0/?back=1&afterReload=1 


From:  Nikolay Pomazan <pomazannn@gmail.com> 		 Сегодня, 7:15 
Subj: Тема помидоров 
Link: https://e.mail.ru/inbox/1:22824ded9dd9d2ef:0/?back=1&afterReload=1 


From:  Nikolay Pomazan <pomazannn@gmail.com> 		 Сегодня, 7:13 
Subj: Тестовое письмо 1 
Link: https://e.mail.ru/inbox/1:2a0ae5e14a1d8eb8:0/?back=1&afterReload=1 


From:  Команда Почты Mail.ru <welcome@corp.mail.ru> 		 Сегодня, 6:57 
Subj: Как воспользоваться почтой с мобильного? 
Link: https://e.mail.ru/inbox/1:192753023b

In [6]:
letters_info

[{'author': 'Nikolay Pomazan <pomazannn@gmail.com>',
  'date': 'Сегодня, 9:30',
  'subj': 'Че не отвечаешь?',
  'link': 'https://e.mail.ru/inbox/1:02380bc5990c721c:0/?back=1&afterReload=1'},
 {'author': 'Nikolay Pomazan <pomazannn@gmail.com>',
  'date': 'Сегодня, 9:29',
  'subj': 'А че если фотку прилепить а а а а?',
  'link': 'https://e.mail.ru/inbox/1:12553ef61196f16d:0/?back=1&afterReload=1'},
 {'author': 'Nikolay Pomazan <pomazannn@gmail.com>',
  'date': 'Сегодня, 7:55',
  'subj': 'Это тема длинного письма',
  'link': 'https://e.mail.ru/inbox/1:3442578eb55fda88:0/?back=1&afterReload=1'},
 {'author': 'Nikolay Pomazan <pomazannn@gmail.com>',
  'date': 'Сегодня, 7:15',
  'subj': 'Тема помидоров',
  'link': 'https://e.mail.ru/inbox/1:22824ded9dd9d2ef:0/?back=1&afterReload=1'},
 {'author': 'Nikolay Pomazan <pomazannn@gmail.com>',
  'date': 'Сегодня, 7:13',
  'subj': 'Тестовое письмо 1',
  'link': 'https://e.mail.ru/inbox/1:2a0ae5e14a1d8eb8:0/?back=1&afterReload=1'},
 {'author': 'Команда

In [7]:
for i in range(len(letters_info)):
    letter_info = letters_info[i]
    driver.get(letter_info["link"])
    wait.until(EC.presence_of_element_located((By.XPATH, "//*[contains(@id, '_BODY')]")))
    letters_info[i]["text"] = driver.find_element_by_xpath("//*[contains(@id, '_BODY')]").text
    driver.back()
    wait.until(EC.title_contains("Входящие"));

In [8]:
letters_info

[{'author': 'Nikolay Pomazan <pomazannn@gmail.com>',
  'date': 'Сегодня, 9:30',
  'subj': 'Че не отвечаешь?',
  'link': 'https://e.mail.ru/inbox/1:02380bc5990c721c:0/?back=1&afterReload=1',
  'text': 'Ты выйграл миллиард. Отвечай уже давай. Пришлешь мне 165тысяч рублей на обслуживание счета и миллиард твой. Все красиво, шито-крыто.'},
 {'author': 'Nikolay Pomazan <pomazannn@gmail.com>',
  'date': 'Сегодня, 9:29',
  'subj': 'А че если фотку прилепить а а а а?',
  'link': 'https://e.mail.ru/inbox/1:12553ef61196f16d:0/?back=1&afterReload=1',
  'text': 'Взял и прикрепил в тело письма картинку'},
 {'author': 'Nikolay Pomazan <pomazannn@gmail.com>',
  'date': 'Сегодня, 7:55',
  'subj': 'Это тема длинного письма',
  'link': 'https://e.mail.ru/inbox/1:3442578eb55fda88:0/?back=1&afterReload=1',
  'text': 'Длинное письмо, очень длинное, оно настолько длинное, что по идее не должно влезть в предварительный просмотр, а должно каким-то образом сократиться, так что его отдельно открывать и вытаскива

In [9]:
mongoclient = MongoClient('mongodb://127.0.0.1:27017')
db = mongoclient['letters']
lettersdb = db.letters

In [10]:
# Drop database:
# mongoclient.drop_database("letters")
lettersdb.insert_many(letters_info)

In [11]:
def show_what_in_database(print_limit=0):
    cursor = lettersdb.find()
    i = 0
    for document in cursor:
        pprint.pprint(document)
        print()
        i += 1
        if print_limit != 0:
            if i == print_limit:
                return

In [12]:
show_what_in_database()

{'_id': ObjectId('5d29447cef3a71879ef0adf0'),
 'author': 'Nikolay Pomazan <pomazannn@gmail.com>',
 'date': 'Сегодня, 9:30',
 'link': 'https://e.mail.ru/inbox/1:02380bc5990c721c:0/?back=1&afterReload=1',
 'subj': 'Че не отвечаешь?',
 'text': 'Ты выйграл миллиард. Отвечай уже давай. Пришлешь мне 165тысяч рублей '
         'на обслуживание счета и миллиард твой. Все красиво, шито-крыто.'}

{'_id': ObjectId('5d29447cef3a71879ef0adf1'),
 'author': 'Nikolay Pomazan <pomazannn@gmail.com>',
 'date': 'Сегодня, 9:29',
 'link': 'https://e.mail.ru/inbox/1:12553ef61196f16d:0/?back=1&afterReload=1',
 'subj': 'А че если фотку прилепить а а а а?',
 'text': 'Взял и прикрепил в тело письма картинку'}

{'_id': ObjectId('5d29447cef3a71879ef0adf2'),
 'author': 'Nikolay Pomazan <pomazannn@gmail.com>',
 'date': 'Сегодня, 7:55',
 'link': 'https://e.mail.ru/inbox/1:3442578eb55fda88:0/?back=1&afterReload=1',
 'subj': 'Это тема длинного письма',
 'text': 'Длинное письмо, очень длинное, оно настолько длинное, что

In [13]:
driver.quit()